In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from flowstock import Akagi, ForwardSimulator

# Set parameter values

In [ ]:
num_cells = 15**2

In [ ]:
n = int(num_cells**0.5)
x = np.linspace(-1, 1, n)
y = np.linspace(-1, 1, n)
xv, yv = np.meshgrid(x, y, sparse=False, indexing='ij')

loc = []
for i in range(n):
    for j in range(n):
        loc.append([x[i], y[j]])

loc = np.array(loc)

In [ ]:
rad = 0.8
N_init = np.exp(-((loc**2).sum(axis=1)**0.5 - rad)**2)
N_init = (N_init * 1000 * N_init.max()).astype(int)
N_init.reshape((n, n))

In [ ]:
# plt.imshow(N_init.reshape((n, n)))
# plt.colorbar()

In [ ]:
s = np.exp((-4 * loc**2).sum(axis=1))
s.reshape((n, n))

In [ ]:
pi = N_init / N_init.max() / 10
pi.reshape((n, n))

In [ ]:
# Array of distances between cells
d = np.zeros((num_cells, num_cells))
for i in range(num_cells):
    for j in range(num_cells):
        d[i, j] = ((loc[i] - loc[j])**2).sum()**0.5

In [ ]:
beta = 1
K = .2

## Generate fake data

In [ ]:
simulator = ForwardSimulator(pi, s, beta, d, K)
simulator.noise_amplitude = 0.0

In [ ]:
num_steps = 11
np.random.seed(0)
N, M_true = simulator.simulate(N_init, num_steps)

In [ ]:
# fig, ax = plt.subplots()

# cs = ax.imshow(N)
# cb = fig.colorbar(cs)

# ax.set_ylabel('t')
# ax.set_xlabel('cell number')

In [ ]:
# fig, ax = plt.subplots()

# ax.plot(N.sum(axis=1), label='total')

# for i in range(N.shape[1]):
#     ax.plot(N[:, i])
    
# ax.set_xlabel('time')
# ax.set_ylabel('population')

# ax.set_yscale('log')

# fig.legend()

## Estimate movement from fake data

In [ ]:
a = Akagi(N, d, K)
a.lamda = 10

a.init_M_moving()

a.M_jitter(1/N.max()/10)

In [ ]:
a.gamma_exc.sum(axis=1)

In [ ]:
a.M.astype(int)[0]

In [ ]:
M_init = a.M.copy()
pi_init = a.pi.copy()
s_init = a.s.copy()
beta_init = a.beta

Pre-compile some numba functions

This wouldn't be done normally, but is interesting when testing

In [ ]:
# from movement import akagi

In [ ]:
# %time akagi._cost(M_init, N)
# %time akagi._cost(M_init, N)

In [ ]:
term_0_log = a.term_0_log(a.pi)
braces = a.term_1_braces(a.pi, a.s, a.beta, a.d)

# %time a.likelihood(a.M, a.pi, a.s, a.beta, term_0_log=term_0_log, term_1_braces=braces)
# %time a.likelihood(a.M, a.pi, a.s, a.beta, term_0_log=term_0_log, term_1_braces=braces)

In [ ]:
%%time
##Use the derivative of the Likelihood function for the optimization of the likelihood function
# %%prun
result = a.exact_inference(1e-4, True)

# Compare output

In [ ]:
M_est = a.M

Does `M` look similar?

In [ ]:
M_true.shape

In [ ]:
M_est.shape

In [ ]:
np.abs((M_true - M_est)).sum()/M_true.sum()

In [ ]:
M_true[0][:10, :10]

In [ ]:
np.rint(M_est[0]).astype(int)[:10, :10]

Are there approximately the right number of people in the end?

In [ ]:
np.rint((M_est.sum(axis=2) - N[:-1])).astype(int)

Have $\pi$,  $s$ and $\beta$ converged well?

In [ ]:
def comp_plot(exact, est, title):
    fig, ax = plt.subplots()
    
    ident = np.linspace(
        min(exact.min(), est.min()),
        max(exact.max(), est.max()),
    )
    ax.plot(ident, ident, alpha=0.5, color='gray')
    
    ax.scatter(
        exact,
        est,
        marker="+"
    )
    
    ax.set_xlabel(r"exact")
    ax.set_ylabel(r"estimated")
    
    ax.set_title(title)
    
    ax.set_aspect('equal')

In [ ]:
comp_plot(pi, a.pi, r"$\pi$")

In [ ]:
_ = plt.hist2d(pi, a.pi, bins=np.linspace(0, 0.2))

In [ ]:
comp_plot(s/s.max(), a.s/a.s.max(), r"$s$")

In [ ]:
comp_plot(M_true.flatten(), M_est.flatten(), r"$M$")

In [ ]:
beta

In [ ]:
a.beta

Save some vars

In [ ]:
np.save("fake/N", N)
np.save("fake/d", d)
np.save("fake/K", K)
np.save("fake/beta", beta)
np.save("fake/s", s)
np.save("fake/pi", pi)
np.save("fake/M_true", M_true)
np.save("fake/M_est", a.M)

In [ ]:
import matplotlib.colors as mcolors

plt.hist2d(
    M_true.flatten(),
    np.rint(M_est.flatten()),
    bins=np.concatenate(([0], np.logspace(-1, 4, 100))),
    norm=mcolors.LogNorm(),
)

plt.plot(
    np.logspace(-1, 4),
    np.logspace(-1, 4),
)

plt.xscale('symlog')
plt.yscale('symlog')

plt.xlabel('true')
plt.ylabel('est')

plt.title("M")

plt.colorbar(label='count')

In [ ]:
np.rint(M_est[0]).astype(int)

In [ ]:
M_true[0]

In [ ]:
beta_array = np.linspace(-4, 4)
L_array_M_est = np.array([a.likelihood(a.M, a.pi, a.s, beta_) for beta_ in beta_array])
L_array_M_true = np.array([a.likelihood(M_true, pi, s, beta_) for beta_ in beta_array])

In [ ]:
plt.plot(beta_array, L_array_M_est, label="est M")
plt.plot(beta_array, L_array_M_true, label="true M, pi, s")
plt.xlabel(r"$\beta$")
plt.ylabel(r"$\mathcal{L}$")
plt.legend()

In [ ]:
plt.imshow(
    a.gamma_exc * M_est[0],
#     norm=mcolors.LogNorm(),
)
plt.colorbar()

In [ ]:
plt.imshow(
    a.gamma_exc * M_true[0],
#     norm=mcolors.LogNorm(),
)
plt.colorbar()

In [ ]:
plt.imshow(
    a.gamma_exc * (M_true - M_est)[0],
#     norm=mcolors.LogNorm(),
)
plt.colorbar()

In [ ]:
N.shape

In [ ]:
np.load("fake/M_true.npy").shape